In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.chdir(r"D:\Random_Datasets\Kaggle\champs-scalar-coupling")
import xgboost as xgb
import pickle
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [11]:
%%time
train_df = pd.read_csv("train_2.csv")
test_df = pd.read_csv("test_2.csv")

Wall time: 2min 27s


In [14]:
test_df.head()

,id,Unnamed: 0,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,abs_distance,x_dist,y_dist,z_dist,x_0_mean,x_0_median,x_0_max,x_0_min,x_0_std,y_0_mean,y_0_median,y_0_max,y_0_min,y_0_std,z_0_mean,z_0_median,z_0_max,z_0_min,z_0_std,abs_distance_mean,abs_distance_median,abs_distance_max,abs_distance_min,abs_distance_std,x_dist_mean,x_dist_median,x_dist_max,x_dist_min,x_dist_std,y_dist_mean,y_dist_median,y_dist_max,y_dist_min,y_dist_std,z_dist_mean,z_dist_median,z_dist_max,z_dist_min,z_dist_std,x_1_mean,x_1_median,x_1_max,x_1_min,x_1_std,y_1_mean,y_1_median,y_1_max,y_1_min,y_1_std,z_1_mean,z_1_median,z_1_max,z_1_min,z_1_std
0,4658147,0,2,0,4,0,-1.661639,0.000000,1.000000,0,0.599539,0.000000,1.000000,2.261178,-2.261178,0.000000,0.000000,0.096261,0.12693,9.38224,-9.234889,1.801936,-0.373901,-0.415277,9.714469,-9.49416,2.108418,0.065339,0.013173,7.637578,-9.134765,1.589036,0.107762,0.038639,7.468888,-7.317726,1.367869,-0.436889,-0.480829,7.52965,-7.824676,1.648573,0.067495,0.011602,6.126566,-8.439682,1.101378,-0.0115,-0.040168,2.453043,-2.453407,1.243605,0.062987,0.039226,2.471449,-2.478801,1.330706,-0.002155,-0.004854,2.443864,-2.452288,1.218074,2.190397,2.183838,2.521012,1.792659,0.085421
1,4658148,1,3,1,4,0,1.661639,0.000000,1.000000,0,-0.599539,0.000000,1.000000,2.261178,2.261178,0.000000,0.000000,0.096261,0.12693,9.38224,-9.234889,1.801936,-0.373901,-0.415277,9.714469,-9.49416,2.108418,0.065339,0.013173,7.637578,-9.134765,1.589036,0.107762,0.038639,7.468888,-7.317726,1.367869,-0.436889,-0.480829,7.52965,-7.824676,1.648573,0.067495,0.011602,6.126566,-8.439682,1.101378,-0.0115,-0.040168,2.453043,-2.453407,1.243605,0.062987,0.039226,2.471449,-2.478801,1.330706,-0.002155,-0.004854,2.443864,-2.452288,1.218074,2.190397,2.183838,2.521012,1.792659,0.085421
2,4658149,2,3,1,4,0,-0.305415,2.017021,0.925332,0,1.302991,0.778866,-0.006178,2.233318,-1.608406,1.238156,0.931511,0.096261,0.12693,9.38224,-9.234889,1.801936,-0.373901,-0.415277,9.714469,-9.49416,2.108418,0.065339,0.013173,7.637578,-9.134765,1.589036,0.107762,0.038639,7.468888,-7.317726,1.367869,-0.436889,-0.480829,7.52965,-7.824676,1.648573,0.067495,0.011602,6.126566,-8.439682,1.101378,-0.0115,-0.040168,2.453043,-2.453407,1.243605,0.062987,0.039226,2.471449,-2.478801,1.330706,-0.002155,-0.004854,2.443864,-2.452288,1.218074,2.190397,2.183838,2.521012,1.792659,0.085421
3,4658150,3,3,2,4,0,-0.305415,2.017021,0.925332,0,0.008672,0.007670,0.002010,2.233532,-0.314086,2.009351,0.923322,0.096261,0.12693,9.38224,-9.234889,1.801936,-0.373901,-0.415277,9.714469,-9.49416,2.108418,0.065339,0.013173,7.637578,-9.134765,1.589036,0.107762,0.038639,7.468888,-7.317726,1.367869,-0.436889,-0.480829,7.52965,-7.824676,1.648573,0.067495,0.011602,6.126566,-8.439682,1.101378,-0.0115,-0.040168,2.453043,-2.453407,1.243605,0.062987,0.039226,2.471449,-2.478801,1.330706,-0.002155,-0.004854,2.443864,-2.452288,1.218074,2.190397,2.183838,2.521012,1.792659,0.085421
4,4658151,4,4,1,4,0,-0.322755,2.026802,-0.893478,0,1.302991,0.778866,-0.006178,2.233315,-1.625746,1.247936,-0.887300,0.096261,0.12693,9.38224,-9.234889,1.801936,-0.373901,-0.415277,9.714469,-9.49416,2.108418,0.065339,0.013173,7.637578,-9.134765,1.589036,0.107762,0.038639,7.468888,-7.317726,1.367869,-0.436889,-0.480829,7.52965,-7.824676,1.648573,0.067495,0.011602,6.126566,-8.439682,1.101378,-0.0115,-0.040168,2.453043,-2.453407,1.243605,0.062987,0.039226,2.471449,-2.478801,1.330706,-0.002155,-0.004854,2.443864,-2.452288,1.218074,2.190397,2.183838,2.521012,1.792659,0.085421


In [15]:
%%time
train_df = train_df.drop(columns = ["Unnamed: 0", "id", "molecule_name"])

Wall time: 1.71 s


In [16]:
train_df.head()

,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,abs_distance,x_dist,y_dist,z_dist,x_0_mean,x_0_median,x_0_max,x_0_min,x_0_std,y_0_mean,y_0_median,y_0_max,y_0_min,y_0_std,z_0_mean,z_0_median,z_0_max,z_0_min,z_0_std,abs_distance_mean,abs_distance_median,abs_distance_max,abs_distance_min,abs_distance_std,x_dist_mean,x_dist_median,x_dist_max,x_dist_min,x_dist_std,y_dist_mean,y_dist_median,y_dist_max,y_dist_min,y_dist_std,z_dist_mean,z_dist_median,z_dist_max,z_dist_min,z_dist_std,x_1_mean,x_1_median,x_1_max,x_1_min,x_1_std,y_1_mean,y_1_median,y_1_max,y_1_min,y_1_std,z_1_mean,z_1_median,z_1_max,z_1_min,z_1_std
0,1,0,0,84.8076,0,0.002150,-0.006031,0.001976,0,-0.012698,1.085804,0.008001,1.091953,0.014849,-1.091835,-0.006025,0.08493,0.058999,9.38224,-9.234889,1.771849,-0.186959,-0.249816,9.714469,-9.49416,2.154356,0.058627,0.021721,7.637578,-9.134765,1.581798,0.095542,0.029502,8.420893,-8.677876,1.455541,-0.199274,-0.280638,8.653665,-8.432506,1.858621,0.061694,0.010513,6.796299,-8.439682,1.233035,-0.010612,-0.057078,1.112979,-1.117153,0.652669,0.012315,0.160798,1.119363,-1.178201,0.573888,-0.003067,-0.000335,1.11588,-1.115035,0.66248,1.0929,1.094027,1.247942,1.060901,0.006819
1,2,0,0,84.8074,0,1.011731,1.463751,0.000277,0,-0.012698,1.085804,0.008001,1.091952,1.024429,0.377947,-0.007724,0.08493,0.058999,9.38224,-9.234889,1.771849,-0.186959,-0.249816,9.714469,-9.49416,2.154356,0.058627,0.021721,7.637578,-9.134765,1.581798,0.095542,0.029502,8.420893,-8.677876,1.455541,-0.199274,-0.280638,8.653665,-8.432506,1.858621,0.061694,0.010513,6.796299,-8.439682,1.233035,-0.010612,-0.057078,1.112979,-1.117153,0.652669,0.012315,0.160798,1.119363,-1.178201,0.573888,-0.003067,-0.000335,1.11588,-1.115035,0.66248,1.0929,1.094027,1.247942,1.060901,0.006819
2,3,0,0,84.8093,0,-0.540815,1.447527,-0.876644,0,-0.012698,1.085804,0.008001,1.091946,-0.528117,0.361722,-0.884645,0.08493,0.058999,9.38224,-9.234889,1.771849,-0.186959,-0.249816,9.714469,-9.49416,2.154356,0.058627,0.021721,7.637578,-9.134765,1.581798,0.095542,0.029502,8.420893,-8.677876,1.455541,-0.199274,-0.280638,8.653665,-8.432506,1.858621,0.061694,0.010513,6.796299,-8.439682,1.233035,-0.010612,-0.057078,1.112979,-1.117153,0.652669,0.012315,0.160798,1.119363,-1.178201,0.573888,-0.003067,-0.000335,1.11588,-1.115035,0.66248,1.0929,1.094027,1.247942,1.060901,0.006819
3,4,0,0,84.8095,0,-0.523814,1.437933,0.906397,0,-0.012698,1.085804,0.008001,1.091948,-0.511115,0.352128,0.898396,0.08493,0.058999,9.38224,-9.234889,1.771849,-0.186959,-0.249816,9.714469,-9.49416,2.154356,0.058627,0.021721,7.637578,-9.134765,1.581798,0.095542,0.029502,8.420893,-8.677876,1.455541,-0.199274,-0.280638,8.653665,-8.432506,1.858621,0.061694,0.010513,6.796299,-8.439682,1.233035,-0.010612,-0.057078,1.112979,-1.117153,0.652669,0.012315,0.160798,1.119363,-1.178201,0.573888,-0.003067,-0.000335,1.11588,-1.115035,0.66248,1.0929,1.094027,1.247942,1.060901,0.006819
4,2,0,0,171.2200,0,-0.027803,2.198949,0.014154,0,-0.013324,1.132466,0.008276,1.066598,-0.014479,1.066484,0.005878,0.08493,0.058999,9.38224,-9.234889,1.771849,-0.186959,-0.249816,9.714469,-9.49416,2.154356,0.058627,0.021721,7.637578,-9.134765,1.581798,0.095542,0.029502,8.420893,-8.677876,1.455541,-0.199274,-0.280638,8.653665,-8.432506,1.858621,0.061694,0.010513,6.796299,-8.439682,1.233035,-0.010612,-0.057078,1.112979,-1.117153,0.652669,0.012315,0.160798,1.119363,-1.178201,0.573888,-0.003067,-0.000335,1.11588,-1.115035,0.66248,1.0929,1.094027,1.247942,1.060901,0.006819


In [17]:
test_df = test_df.drop(columns = ["Unnamed: 0"])
test_df.head()

,id,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,abs_distance,x_dist,y_dist,z_dist,x_0_mean,x_0_median,x_0_max,x_0_min,x_0_std,y_0_mean,y_0_median,y_0_max,y_0_min,y_0_std,z_0_mean,z_0_median,z_0_max,z_0_min,z_0_std,abs_distance_mean,abs_distance_median,abs_distance_max,abs_distance_min,abs_distance_std,x_dist_mean,x_dist_median,x_dist_max,x_dist_min,x_dist_std,y_dist_mean,y_dist_median,y_dist_max,y_dist_min,y_dist_std,z_dist_mean,z_dist_median,z_dist_max,z_dist_min,z_dist_std,x_1_mean,x_1_median,x_1_max,x_1_min,x_1_std,y_1_mean,y_1_median,y_1_max,y_1_min,y_1_std,z_1_mean,z_1_median,z_1_max,z_1_min,z_1_std
0,4658147,2,0,4,0,-1.661639,0.000000,1.000000,0,0.599539,0.000000,1.000000,2.261178,-2.261178,0.000000,0.000000,0.096261,0.12693,9.38224,-9.234889,1.801936,-0.373901,-0.415277,9.714469,-9.49416,2.108418,0.065339,0.013173,7.637578,-9.134765,1.589036,0.107762,0.038639,7.468888,-7.317726,1.367869,-0.436889,-0.480829,7.52965,-7.824676,1.648573,0.067495,0.011602,6.126566,-8.439682,1.101378,-0.0115,-0.040168,2.453043,-2.453407,1.243605,0.062987,0.039226,2.471449,-2.478801,1.330706,-0.002155,-0.004854,2.443864,-2.452288,1.218074,2.190397,2.183838,2.521012,1.792659,0.085421
1,4658148,3,1,4,0,1.661639,0.000000,1.000000,0,-0.599539,0.000000,1.000000,2.261178,2.261178,0.000000,0.000000,0.096261,0.12693,9.38224,-9.234889,1.801936,-0.373901,-0.415277,9.714469,-9.49416,2.108418,0.065339,0.013173,7.637578,-9.134765,1.589036,0.107762,0.038639,7.468888,-7.317726,1.367869,-0.436889,-0.480829,7.52965,-7.824676,1.648573,0.067495,0.011602,6.126566,-8.439682,1.101378,-0.0115,-0.040168,2.453043,-2.453407,1.243605,0.062987,0.039226,2.471449,-2.478801,1.330706,-0.002155,-0.004854,2.443864,-2.452288,1.218074,2.190397,2.183838,2.521012,1.792659,0.085421
2,4658149,3,1,4,0,-0.305415,2.017021,0.925332,0,1.302991,0.778866,-0.006178,2.233318,-1.608406,1.238156,0.931511,0.096261,0.12693,9.38224,-9.234889,1.801936,-0.373901,-0.415277,9.714469,-9.49416,2.108418,0.065339,0.013173,7.637578,-9.134765,1.589036,0.107762,0.038639,7.468888,-7.317726,1.367869,-0.436889,-0.480829,7.52965,-7.824676,1.648573,0.067495,0.011602,6.126566,-8.439682,1.101378,-0.0115,-0.040168,2.453043,-2.453407,1.243605,0.062987,0.039226,2.471449,-2.478801,1.330706,-0.002155,-0.004854,2.443864,-2.452288,1.218074,2.190397,2.183838,2.521012,1.792659,0.085421
3,4658150,3,2,4,0,-0.305415,2.017021,0.925332,0,0.008672,0.007670,0.002010,2.233532,-0.314086,2.009351,0.923322,0.096261,0.12693,9.38224,-9.234889,1.801936,-0.373901,-0.415277,9.714469,-9.49416,2.108418,0.065339,0.013173,7.637578,-9.134765,1.589036,0.107762,0.038639,7.468888,-7.317726,1.367869,-0.436889,-0.480829,7.52965,-7.824676,1.648573,0.067495,0.011602,6.126566,-8.439682,1.101378,-0.0115,-0.040168,2.453043,-2.453407,1.243605,0.062987,0.039226,2.471449,-2.478801,1.330706,-0.002155,-0.004854,2.443864,-2.452288,1.218074,2.190397,2.183838,2.521012,1.792659,0.085421
4,4658151,4,1,4,0,-0.322755,2.026802,-0.893478,0,1.302991,0.778866,-0.006178,2.233315,-1.625746,1.247936,-0.887300,0.096261,0.12693,9.38224,-9.234889,1.801936,-0.373901,-0.415277,9.714469,-9.49416,2.108418,0.065339,0.013173,7.637578,-9.134765,1.589036,0.107762,0.038639,7.468888,-7.317726,1.367869,-0.436889,-0.480829,7.52965,-7.824676,1.648573,0.067495,0.011602,6.126566,-8.439682,1.101378,-0.0115,-0.040168,2.453043,-2.453407,1.243605,0.062987,0.039226,2.471449,-2.478801,1.330706,-0.002155,-0.004854,2.443864,-2.452288,1.218074,2.190397,2.183838,2.521012,1.792659,0.085421


In [18]:
print("train_df = {}".format(train_df.shape))
print("test_df = {}".format(test_df.shape))

train_df = (4658147, 66)
test_df = (2505542, 66)


## Feature Creation

We will expand our feature set by creating more features based on distances. Following are the features, we will create for next iteration of the model training:
1. *Statistical summary differences* - Subtract x_0 from the derived features. The new feature created will be x_0_mean_diff, x_0_kurtosis_diff, x_0_max_diff, x_0_min_diff and x_0_std_diff. Delete the parent columns afterwards.
2. Repeat this exercise for
    - for other atomic features like y_0, z_0 etc.
    - for abs_distance
    - for x_dist, y_dist, z_dist
3. Angle Based features

In [19]:
def diff_columns(dataframe, col_family):
    """
    this functions takes the dataframe and col_family as input and return the dataframe
    where differenced features are returned.
    """
    df = dataframe
    col_list = [col for col in df.columns if col.startswith(col_family)]
    
    for i in col_list[1:]:
        df[i + "_diff"] = df[i] - df[col_list[0]]
        
    return df

In [20]:
%%time
col_family = ["x_0", "y_0", "z_0", "x_1", "y_1", "z_1", "abs_distance", "x_dist", "y_dist", "z_dist"]

for i in col_family:
    train_df = diff_columns(train_df, i)

Wall time: 1.06 s


In [21]:
train_df.head()

,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,abs_distance,x_dist,y_dist,z_dist,x_0_mean,x_0_median,x_0_max,x_0_min,x_0_std,y_0_mean,y_0_median,y_0_max,y_0_min,y_0_std,z_0_mean,z_0_median,z_0_max,z_0_min,z_0_std,abs_distance_mean,abs_distance_median,abs_distance_max,abs_distance_min,abs_distance_std,x_dist_mean,x_dist_median,x_dist_max,x_dist_min,x_dist_std,y_dist_mean,y_dist_median,y_dist_max,y_dist_min,y_dist_std,z_dist_mean,z_dist_median,z_dist_max,z_dist_min,...,x_0_mean_diff,x_0_median_diff,x_0_max_diff,x_0_min_diff,x_0_std_diff,y_0_mean_diff,y_0_median_diff,y_0_max_diff,y_0_min_diff,y_0_std_diff,z_0_mean_diff,z_0_median_diff,z_0_max_diff,z_0_min_diff,z_0_std_diff,x_1_mean_diff,x_1_median_diff,x_1_max_diff,x_1_min_diff,x_1_std_diff,y_1_mean_diff,y_1_median_diff,y_1_max_diff,y_1_min_diff,y_1_std_diff,z_1_mean_diff,z_1_median_diff,z_1_max_diff,z_1_min_diff,z_1_std_diff,abs_distance_mean_diff,abs_distance_median_diff,abs_distance_max_diff,abs_distance_min_diff,abs_distance_std_diff,x_dist_mean_diff,x_dist_median_diff,x_dist_max_diff,x_dist_min_diff,x_dist_std_diff,y_dist_mean_diff,y_dist_median_diff,y_dist_max_diff,y_dist_min_diff,y_dist_std_diff,z_dist_mean_diff,z_dist_median_diff,z_dist_max_diff,z_dist_min_diff,z_dist_std_diff
0,1,0,0,84.8076,0,0.002150,-0.006031,0.001976,0,-0.012698,1.085804,0.008001,1.091953,0.014849,-1.091835,-0.006025,0.08493,0.058999,9.38224,-9.234889,1.771849,-0.186959,-0.249816,9.714469,-9.49416,2.154356,0.058627,0.021721,7.637578,-9.134765,1.581798,0.095542,0.029502,8.420893,-8.677876,1.455541,-0.199274,-0.280638,8.653665,-8.432506,1.858621,0.061694,0.010513,6.796299,-8.439682,1.233035,-0.010612,-0.057078,1.112979,-1.117153,...,0.082779,0.056849,9.380090,-9.237039,1.769698,-0.180928,-0.243785,9.720501,-9.488128,2.160387,0.056651,0.019745,7.635602,-9.136741,1.579822,0.025013,0.173496,1.132061,-1.165503,0.586586,-1.088871,-1.086140,0.030076,-2.200839,-0.423324,1.084899,1.086026,1.239941,1.052900,-0.001182,-0.996411,-1.062451,7.328940,-9.769829,0.363588,-0.214122,-0.295486,8.638816,-8.447354,1.843773,1.153529,1.102349,7.888135,-7.347847,2.324870,-0.004587,-0.051053,1.119004,-1.111128,0.658694
1,2,0,0,84.8074,0,1.011731,1.463751,0.000277,0,-0.012698,1.085804,0.008001,1.091952,1.024429,0.377947,-0.007724,0.08493,0.058999,9.38224,-9.234889,1.771849,-0.186959,-0.249816,9.714469,-9.49416,2.154356,0.058627,0.021721,7.637578,-9.134765,1.581798,0.095542,0.029502,8.420893,-8.677876,1.455541,-0.199274,-0.280638,8.653665,-8.432506,1.858621,0.061694,0.010513,6.796299,-8.439682,1.233035,-0.010612,-0.057078,1.112979,-1.117153,...,-0.926801,-0.952732,8.370510,-10.246620,0.760118,-1.650710,-1.713567,8.250718,-10.957911,0.690605,0.058351,0.021444,7.637301,-9.135041,1.581521,0.025013,0.173496,1.132061,-1.165503,0.586586,-1.088871,-1.086140,0.030076,-2.200839,-0.423324,1.084899,1.086026,1.239941,1.052900,-0.001182,-0.996410,-1.062450,7.328941,-9.769827,0.363589,-1.223703,-1.305067,7.629236,-9.456935,0.834192,-0.316253,-0.367434,6.418352,-8.817629,0.855088,-0.002888,-0.049353,1.120704,-1.109429,0.660394
2,3,0,0,84.8093,0,-0.540815,1.447527,-0.876644,0,-0.012698,1.085804,0.008001,1.091946,-0.528117,0.361722,-0.884645,0.08493,0.058999,9.38224,-9.234889,1.771849,-0.186959,-0.249816,9.714469,-9.49416,2.154356,0.058627,0.021721,7.637578,-9.134765,1.581798,0.095542,0.029502,8.420893,-8.677876,1.455541,-0.199274,-0.280638,8.653665,-8.432506,1.858621,0.061694,0.010513,6.796299,-8.439682,1.233035,-0.010612,-0.057078,1.112979,-1.117153,...,0.625745,0.599814,9.923056,-8.694074,2.312664,-1.634485,-1.697343,8.266943,-10.941686,0.706830,0.935271,0.898365,8.514222,-8.258121,2.458441,0.025013,0.173496,1.132061,-1.165503,0.586586,-1.088871,-1.086140,0.030076,-2.200839,-0.423324,1.084899,1.086026,1.239941,1.052900,-0.001182,-0.996405,-1.062445,7.328946,-9.769822,0.363595,0.328843,0.247479,9.181782,-7.904389,2.386738,-0.300029,-0.351209,6.434577,-8.801405,0.871312,0.874033,0.827567,1.997624,

In [22]:
train_df.shape

(4658147, 116)

In [23]:
# similarly implementing for test_df
for i in col_family:
    test_df = diff_columns(test_df, i)
    
test_df.shape

(2505542, 116)

In [24]:
%%time
drop_cols = [col for col in train_df.columns if col.endswith("mean") or col.endswith("median") or col.endswith("max") or 
             col.endswith("min") or col.endswith("std")]

cols = [col for col in train_df.columns if col not in drop_cols]
train_df = train_df[cols]

# removing target columns
cols.remove("scalar_coupling_constant")
test_df = test_df[cols]

Wall time: 21.6 s


In [25]:
print("train_df = {}".format(train_df.shape))
print("test_df = {}".format(test_df.shape))

train_df = (4658147, 66)
test_df = (2505542, 65)


In [27]:
test = pd.read_csv("test.csv")
id_ = list(test["id"])
test_df.insert(0, "id", id_)

In [28]:
%%time
train_df.to_csv("train_3.csv", index = False)
test_df.to_csv("test_3.csv", index = False)

Wall time: 12min 36s


## Train the Model

In [15]:
%%time
x_train, x_valid, y_train, y_valid = train_test_split(train_df.drop(columns = ["scalar_coupling_constant"]), 
                                                      train_df["scalar_coupling_constant"], 
                                                      test_size = 0.2, 
                                                      random_state = 42)

Wall time: 6.35 s


In [16]:
%%time
# converting the dataset into DMatrices
dtrain = xgb.DMatrix(x_train, label = y_train)
dvalid = xgb.DMatrix(x_valid, label = y_valid)

Wall time: 9.27 s


In [18]:
%%time
# training the Model
params = {
    "eta":0.1,
    "alpha": 0.1,
    "max_leaves": 128,
    "max_depth": 9,
    "n_estimators": 100,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 1,
    "gamma": 0,
    "eval_metric": "mae",
    "nthreads": 4,
    "objective": "reg:linear",
    "silent": 1
}

model_train = xgb.train(params, dtrain, 2500, evals = [(dvalid, "valid_set")], verbose_eval = 100, early_stopping_rounds = 40)

[0]	valid_set-mae:16.4556
Will train until valid_set-mae hasn't improved in 40 rounds.
[100]	valid_set-mae:2.34673
[200]	valid_set-mae:2.27583
[300]	valid_set-mae:2.23963
[400]	valid_set-mae:2.21079
[500]	valid_set-mae:2.18997
[600]	valid_set-mae:2.17385
[700]	valid_set-mae:2.15951
[800]	valid_set-mae:2.1487
[900]	valid_set-mae:2.13862
[1000]	valid_set-mae:2.13068
[1100]	valid_set-mae:2.12375
[1200]	valid_set-mae:2.11658
[1300]	valid_set-mae:2.10975
[1400]	valid_set-mae:2.10354
[1500]	valid_set-mae:2.09814
[1600]	valid_set-mae:2.09341
[1700]	valid_set-mae:2.08914
[1800]	valid_set-mae:2.08569
[1900]	valid_set-mae:2.08131
[2000]	valid_set-mae:2.07694
[2100]	valid_set-mae:2.07248
[2200]	valid_set-mae:2.06881
[2300]	valid_set-mae:2.06556
[2400]	valid_set-mae:2.06228
[2499]	valid_set-mae:2.05906
Wall time: 9h 32min 21s


In [20]:
pickle.dump(model_train, open('pmp_xgb_2.pickle.dat','wb'))

In [21]:
dtest = xgb.DMatrix(test_df)

In [22]:
%%time
y_pred = model_train.predict(dtest)

Wall time: 1min 25s


In [27]:
result_df = pd.DataFrame({
    "id": id_,
    "scalar_coupling_constant": y_pred
})

In [28]:
result_df.sample(10)

,id,scalar_coupling_constant
1262134,5920281,4.565378
980359,5638506,86.819473
1298038,5956185,3.487503
22265,4680412,-0.557471
1144444,5802591,5.276308
1156219,5814366,6.848283
556153,5214300,1.720493
2243775,6901922,-0.189835
2316645,6974792,-9.057570
497695,5155842,-2.098878


In [29]:
result_df.to_csv("submission_02.csv", index = False)